In [7]:
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [8]:
# load the saved model
model = load_model('my_model.h5')

In [9]:
N = 200
x_test = np.zeros((N, 50, 50, 1), 'float')
y_test = np.zeros((N), 'float')

count = 0
class_labels = sorted(os.listdir("Dataset/test"))
for i, class_label in enumerate(class_labels):
    image_files = sorted(os.listdir(os.path.join("Dataset/test", class_label)))
    for image_file in image_files:
        im = cv2.imread(os.path.join("Dataset/test", class_label, image_file), cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (50, 50))
        x_test[count, :, :, 0] = im / 255.
        y_test[count] = i
        count += 1

y_test = to_categorical(y_test)
print('x_test', x_test.shape)
print('y_test', y_test.shape)

x_test (200, 50, 50, 1)
y_test (200, 4)


In [10]:
# use the model to make predictions
predictions = model.predict(x_test)

# Get the class predictions and confidence scores
class_preds = predictions.argmax(axis=1)
confidences = np.max(predictions, axis=-1)

# Define the class names
class_names = ['B', 'D', 'R', 'S']

# Define the test data directory
test_dir = 'Dataset/test/'

test_images = []
for class_folder in os.listdir(test_dir):
    class_folder_path = os.path.join(test_dir, class_folder)
    if os.path.isdir(class_folder_path):
        for image_file in os.listdir(class_folder_path):
            if image_file.endswith('.jpg'):
                image_path = os.path.join(class_folder_path, image_file)
                test_images.append(image_path)

7/7 [==============================] - 0s 5ms/step


In [13]:
# Save the predictions to a file
with open('result.txt', 'w') as f:
    for i in range(len(test_images)):
        image_name = os.path.basename(test_images[i])
        class_name = class_names[class_preds[i]]
        confidence = confidences[i]
        f.write("{}: ,{}\n".format(
            image_name, class_name))

In [14]:
# evaluate the model on test data
score = model.evaluate(x_test, y_test)

# print the model's accuracy
print('Test accuracy:', score[1])

7/7 [==============================] - 0s 6ms/step - loss: 0.8702 - accuracy: 0.6600
Test accuracy: 0.6600000262260437
